## 英译中

In [ ]:
def translate_FinanceBench(from_dir, to_dir, llm_config):
    import pandas as pd
    import os
    from openai import OpenAI
    from functools import partial
    import sys; sys.path.append("..")
    from utils.spliters import init_spliter
    from minirag.utils import compute_mdhash_id

    client = OpenAI(base_url=llm_config['base_url'],api_key=llm_config["api_key"])
    gen_resp = partial(client.chat.completions.create,model=llm_config['model'],temperature=0.1, top_p=1, max_tokens=1000,)

    ## translate_text：翻译指定文本
    def translate_text(text):
        response = gen_resp(messages=[dict(role="system",content="将文字翻译成中文，直接输出翻译结果"),dict(role="user",content=text)])
        return response.choices[0].message.content  

    os.makedirs(to_dir,exist_ok=True)
    query_df = pd.read_json(os.path.join(from_dir,"queries.jsonl"),lines=True)
    query_df["text_zh"] = query_df["text"].apply(translate_text)
    query_df.to_json(os.path.join(to_dir, "queries.json"), orient="records", indent=4, force_ascii=False)
    corpus_df = pd.read_json(os.path.join(from_dir,"corpus.jsonl"),lines=True)
    corpus_df["text_zh"] = corpus_df["text"].apply(translate_text)
    corpus_df.to_json(os.path.join(to_dir, "corpus.json"), orient="records", indent=4, force_ascii=False)
    


In [ ]:
translate_FinanceBench(
    "resources/data/_raw/FinanceBench",
    "resources/data/ablation_rag/financebench_zh/0_origin",
    dict(model="judger", base_url="http://localhost:12235/v1",api_key="empty")
)

## 文档主题分割

In [ ]:
def split_corpus(from_dir, to_dir, spliter_config):
    import pandas as pd
    import os
    import sys; sys.path.append("..")
    from utils.spliters import init_spliter
    import json
    import shutil

    # 拷贝原目录到新目录
    shutil.copytree(from_dir, to_dir,dirs_exist_ok=True)

    # 加载原始数据
    corpus_df = pd.read_json(os.path.join(to_dir, "corpus.json"))
    query_df = pd.read_json(os.path.join(to_dir, "queries.json"))

    # 初始化分词器
    spliter = init_spliter(**spliter_config)

    chunk_list = []

    for idx, row in corpus_df.iterrows():
        sentence_df = spliter.split_text_to_sentences(row['text_zh'])
        sentence_df = spliter.add_buffered_sentences(sentence_df)
        chunk_df = spliter.cluster(sentence_df)

        for _, crow in chunk_df.iterrows():
            chunk_text = crow['chunk']
            chunk_list.append({
                "doc_id": row["_id"],
                "doc_text": row['text_zh'],
                "chunk_text": chunk_text
            })

    # 保存 chunk.json
    with open(os.path.join(to_dir, "chunk.json"), "w", encoding="utf-8") as f:
        json.dump(chunk_list, f, ensure_ascii=False, indent=4)


    print(f"切分完成，共生成 {len(chunk_list)} 个 chunk。queries.json 和 corpus.json 也已保存到 {to_dir}")

In [ ]:
split_corpus(
    "resources/data/ablation_rag/financebench_zh/0_origin",
    "resources/data/ablation_rag/financebench_zh/seq",
    dict(method="doc_seq_model_spliter", model_path="resources/open_models/nlp_bert_document-segmentation_chinese-base"),
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


切分完成，共生成 301 个 chunk。queries.json 和 corpus.json 也已保存到 resources/data/ablation_rag/FinanceBench_zh/seq


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
split_corpus(
    "resources/data/ablation_rag/financebench_zh/0_origin",
    "resources/data/ablation_rag/financebench_zh/sim",
    dict(method="cos_sim_spliter", model_path="resources/open_models/bge-large-zh-v1.5")
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Chunks:   0%|          | 0/2 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Chunks:   0%|          | 0/3 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a cal

切分完成，共生成 137 个 chunk。queries.json 和 corpus.json 也已保存到 resources/data/ablation_rag/FinanceBench_zh/sim


## 片段摘要生成

In [ ]:
async def summarize_chunks(from_dir, to_dir, llm_config=None):
    import json
    import os
    import asyncio
    import shutil
    from openai import AsyncOpenAI
    from minirag.utils import compute_mdhash_id

    shutil.copytree(from_dir, to_dir,dirs_exist_ok=True)

    # 初始化 OpenAI 客户端
    client = AsyncOpenAI(base_url=llm_config['base_url'], api_key=llm_config["api_key"]) if llm_config else None
    semaphore = asyncio.Semaphore(4)

    async def summarize_text(text):
        async with semaphore:
            try:
                response = await client.chat.completions.create(
                    model=llm_config['model'],
                    messages=[
                        {"role": "system", "content": "为以下内容生成摘要，直接输出结果"},
                        {"role": "user", "content": text}
                    ],
                    temperature=0.1,
                    top_p=1,
                    max_tokens=1000
                )
                return response.choices[0].message.content
            except Exception as e:
                print(f"摘要失败: {e}")
                return text[:50]

    # 读取 chunk.json
    with open(os.path.join(to_dir, "chunk.json"), "r", encoding="utf-8") as f:
        chunk_data = json.load(f)

    tasks = []
    metadata = []

    for item in chunk_data:
        chunk_text = item["chunk_text"]
        if len(chunk_text) < 50 or client is None:
            future = asyncio.Future()
            future.set_result(chunk_text)
            tasks.append(future)
        else:
            tasks.append(summarize_text(chunk_text))
        metadata.append(item)

    summaries = await asyncio.gather(*tasks)

    chunk_sum_list = []
    for meta, summary in zip(metadata, summaries):
        chunk_sum_list.append({
            "doc_id": meta["doc_id"],
            "chunk_id": compute_mdhash_id(summary.strip(), prefix="chunk-"),
            "doc_text": meta["doc_text"],
            "chunk_text": meta["chunk_text"],
            "chunk_sum_text": summary,
        })

    with open(os.path.join(to_dir, "chunk_sum.json"), "w", encoding="utf-8") as f:
        json.dump(chunk_sum_list, f, ensure_ascii=False, indent=4)

    print(f"摘要完成，共处理 {len(chunk_sum_list)} 个 chunk。所有数据保存在 {to_dir}")


In [ ]:
await summarize_chunks(
    "resources/data/ablation_rag/financebench_zh/seq",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum",
    dict(model="judger", base_url="http://localhost:12235/v1",api_key="empty"),
)


/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12235/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http:/

In [ ]:
await summarize_chunks(
    "resources/data/ablation_rag/financebench_zh/sim",
    "resources/data/ablation_rag/financebench_zh/sim@q25_b14_sum",
    dict(model="judger", base_url="http://localhost:12235/v1",api_key="empty"),
)

INFO:datasets:PyTorch version 2.6.0 available.
INFO:datasets:Polars version 1.29.0 available.
INFO:FlagEmbedding.abc.inference.AbsEmbedder:Start multi-process pool on devices: cuda:0, cuda:1, cuda:2, cuda:3, cuda:4, cuda:5, cuda:6, cuda:7
Chunks:   0%|          | 0/2 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Chunks:   0%|          | 0/3 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded enc

In [ ]:
await summarize_chunks(
    "resources/data/ablation_rag/financebench_zh/seq",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b3_sft_sum",
    dict(model="lora", base_url="http://localhost:12239/v1",api_key="empty"),
)

INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"

摘要完成，共处理 301 个 chunk。所有数据保存在 resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum


In [ ]:
await summarize_chunks(
    "resources/data/ablation_rag/financebench_zh/seq",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b3_sft_sum",
    dict(model="lora", base_url="http://localhost:12239/v1",api_key="empty"),
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http:/

In [ ]:
await summarize_chunks(
    "resources/data/ablation_rag/financebench_zh/seq",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b3_base_sum",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"

摘要完成，共处理 301 个 chunk。所有数据保存在 resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum


In [5]:
await summarize_chunks(
    "resources/data/ablation_rag/FinanceBench_zh/sim",
    "resources/data/ablation_rag/FinanceBench_zh/sim@q25_b3_base_sum",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"

摘要完成，共处理 137 个 chunk。所有数据保存在 resources/data/ablation_rag/FinanceBench_zh/sim@q25_b3_base_sum


## 构建图索引

In [1]:
async def build_rag_index(from_dir, to_dir, embed_model_path, rag_llm_config):
    import os
    import shutil
    import pandas as pd
    import logging
    from functools import partial
    from transformers import AutoTokenizer, AutoModel
    from minirag import MiniRAG
    from minirag.utils import EmbeddingFunc
    from minirag.llm import openai_complete_if_cache, hf_embedding
    from minirag.prompt import PROMPTS

    import sys; sys.path.append("..")
    from utils.rag import prompts as custom_prompts

    # ✅ 更新 Prompt 词库
    PROMPTS.update(custom_prompts)

    # ✅ 日志设置
    logging.basicConfig(format="%(levelname)s: %(message)s", level=logging.INFO)

    # ✅ 拷贝目录（合并）
    shutil.copytree(from_dir, to_dir,dirs_exist_ok=True)

    # ✅ 准备路径和目录
    rag_data_dir = os.path.join(to_dir, "rag_data")
    os.makedirs(rag_data_dir,exist_ok=True)

    # ✅ 加载嵌入模型
    embed_tokenizer = AutoTokenizer.from_pretrained(embed_model_path, model_max_length=512)
    embed_model = AutoModel.from_pretrained(embed_model_path)

    # ✅ 初始化 MiniRAG
    rag = MiniRAG(
        working_dir=rag_data_dir,
        llm_model_func=lambda prompt, **kwargs: openai_complete_if_cache(
            prompt=prompt, **rag_llm_config, **kwargs
        ),
        llm_model_max_token_size=500,
        llm_model_name=rag_llm_config["model"],
        embedding_func=EmbeddingFunc(
            embedding_dim=embed_model.config.hidden_size,
            max_token_size=embed_model.config.max_position_embeddings,
            func=partial(hf_embedding, embed_model=embed_model, tokenizer=embed_tokenizer),
        )
    )

    # ✅ 加载 chunk summary 文本
    chunk_sum_path = os.path.join(to_dir, "chunk_sum.json")
    chunk_sum_df = pd.read_json(chunk_sum_path)
    chunk_texts = chunk_sum_df["chunk_sum_text"].tolist()

    # ✅ 多条异步插入
    batch_size=4
    for i in range(0, len(chunk_texts), batch_size):
        batch = chunk_texts[i:i+batch_size]
        await rag.ainsert(batch)

    print(f"✅ 异步 RAG 索引构建完成，共插入 {len(chunk_texts)} 个 chunk。索引保存在：{rag_data_dir}")
    return rag

In [ ]:
await build_rag_index(
    "resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum",
    "resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@q25_b3_base_index",
    "resources/open_models/bge-large-zh-v1.5",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:minirag:Logger initialized for working directory: resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@25_b3_base_index/rag_data
INFO:minirag:Load KV llm_response_cache with 585 data
INFO:minirag:Load KV full_docs with 291 data
INFO:minirag:Load KV text_chunks with 292 data
INFO:minirag:Loaded graph from resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@25_b3_base_index/rag_data/graph_chunk_entity_relation.graphml with 686 nodes, 364 edges
INFO:nano-vectordb:Load (633, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@25_b3_base_index/rag_data/vdb_entities.json'} 633 data
INFO

✅ 异步 RAG 索引构建完成，共插入 301 个 chunk。索引保存在：resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@25_b3_base_index/rag_data


MiniRAG(working_dir='resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_sft_sum@25_b3_base_index/rag_data', kv_storage='JsonKVStorage', vector_storage='NanoVectorDBStorage', graph_storage='NetworkXStorage', log_level=0, chunk_token_size=1200, chunk_overlap_token_size=100, tiktoken_model_name='gpt-4o-mini', entity_extract_max_gleaning=1, entity_summary_to_max_tokens=500, node_embedding_algorithm='node2vec', node2vec_params={'dimensions': 1536, 'num_walks': 10, 'walk_length': 40, 'window_size': 2, 'iterations': 3, 'random_seed': 3}, embedding_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x773d7da22e60>, embedding_batch_num=32, embedding_func_max_async=16, llm_model_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x773d75140a60>, llm_model_name='base', llm_model_max_token_size=1000, llm_model_max_async=16, llm_model_kwargs={}, vector_db_storage_cls_kwargs={}, enable_llm_cache=True, addon_params={}, convert_response_to_

In [2]:
await build_rag_index(
    "resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b3_base_sum@q25_b3_base_index",
    "resources/open_models/bge-large-zh-v1.5",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:minirag:Logger initialized for working directory: resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum@q25_b3_base_index/rag_data
INFO:minirag:Load KV llm_response_cache with 189 data
INFO:minirag:Load KV full_docs with 91 data
INFO:minirag:Load KV text_chunks with 91 data
INFO:minirag:Loaded graph from resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum@q25_b3_base_index/rag_data/graph_chunk_entity_relation.graphml with 470 nodes, 240 edges
INFO:nano-vectordb:Load (448, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum@q25_b3_base_index/rag_data/vdb_entities.json'} 448 data


INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 16 vectors to entities


INFO:minirag:Inserting 16 vectors to entities_name
INFO:minirag:Inserting 10 vectors to relationships
INFO:minirag:Writing graph with 483 nodes, 250 edges
INFO:minirag:[New Docs] inserting 2 docs
INFO:minirag:[New Chunks] inserting 2 chunks
INFO:minirag:Inserting 2 vectors to chunks
INFO:minirag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:minirag:Inserting 37 vectors to entities_name
INFO:minirag:Inserting 29 vectors to relationships
INFO:minirag:Writing graph with 1222 nodes, 728 edges
INFO:minirag:[New Docs] inserting 1 docs
INFO:minirag:[New Chunks] inserting 1 chunks
INFO:minirag:Inserting 1 vectors to chunks
INFO:minirag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 5 vectors to entities
INFO:minirag:Inserting 5 vectors to entities_name
INFO:minirag:Inserting 4 vectors to relationships


INFO:minirag:Writing graph with 1225 nodes, 732 edges


✅ 异步 RAG 索引构建完成，共插入 301 个 chunk。索引保存在：resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum@q25_b3_base_index/rag_data


MiniRAG(working_dir='resources/data/ablation_rag/FinanceBench_zh/seq@q25_b3_base_sum@q25_b3_base_index/rag_data', kv_storage='JsonKVStorage', vector_storage='NanoVectorDBStorage', graph_storage='NetworkXStorage', log_level=0, chunk_token_size=1200, chunk_overlap_token_size=100, tiktoken_model_name='gpt-4o-mini', entity_extract_max_gleaning=1, entity_summary_to_max_tokens=500, node_embedding_algorithm='node2vec', node2vec_params={'dimensions': 1536, 'num_walks': 10, 'walk_length': 40, 'window_size': 2, 'iterations': 3, 'random_seed': 3}, embedding_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x74005509ef80>, embedding_batch_num=32, embedding_func_max_async=16, llm_model_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x74004d3c2560>, llm_model_name='base', llm_model_max_token_size=1000, llm_model_max_async=16, llm_model_kwargs={}, vector_db_storage_cls_kwargs={}, enable_llm_cache=True, addon_params={}, convert_response_t

In [2]:
await build_rag_index(
    "resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum",
    "resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index",
    "resources/open_models/bge-large-zh-v1.5",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

/mnt/disk2/xinghua.jia/workspace/finbot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:minirag:Logger initialized for working directory: resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index/rag_data
INFO:minirag:Load KV llm_response_cache with 528 data
INFO:minirag:Load KV full_docs with 264 data
INFO:minirag:Load KV text_chunks with 264 data
INFO:minirag:Loaded graph from resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index/rag_data/graph_chunk_entity_relation.graphml with 982 nodes, 609 edges
INFO:nano-vectordb:Load (939, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index/rag_data/vdb_entities.json'} 939 data
INFO:nano-

INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 19 vectors to entities


...

INFO:minirag:Inserting 30 vectors to entities_name
INFO:minirag:Inserting 14 vectors to relationships
INFO:minirag:Writing graph with 1106 nodes, 673 edges
INFO:minirag:[New Docs] inserting 1 docs
INFO:minirag:[New Chunks] inserting 1 chunks
INFO:minirag:Inserting 1 vectors to chunks
INFO:minirag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 2 vectors to entities
INFO:minirag:Inserting 2 vectors to entities_name
INFO:minirag:Inserting 1 vectors to relationships


INFO:minirag:Writing graph with 1107 nodes, 674 edges


✅ 异步 RAG 索引构建完成，共插入 301 个 chunk。索引保存在：resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index/rag_data


MiniRAG(working_dir='resources/data/ablation_rag/financebench_zh/seq@q25_b14_sum@q25_b3_base_index/rag_data', kv_storage='JsonKVStorage', vector_storage='NanoVectorDBStorage', graph_storage='NetworkXStorage', log_level=0, chunk_token_size=1200, chunk_overlap_token_size=100, tiktoken_model_name='gpt-4o-mini', entity_extract_max_gleaning=1, entity_summary_to_max_tokens=500, node_embedding_algorithm='node2vec', node2vec_params={'dimensions': 1536, 'num_walks': 10, 'walk_length': 40, 'window_size': 2, 'iterations': 3, 'random_seed': 3}, embedding_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x7dff86632e60>, embedding_batch_num=32, embedding_func_max_async=16, llm_model_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x7dff7e1fa290>, llm_model_name='base', llm_model_max_token_size=500, llm_model_max_async=16, llm_model_kwargs={}, vector_db_storage_cls_kwargs={}, enable_llm_cache=True, addon_params={}, convert_response_to_jso

In [4]:
await build_rag_index(
    "resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum",
    "resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index",
    "resources/open_models/bge-large-zh-v1.5",
    dict(model="base", base_url="http://localhost:12239/v1",api_key="empty"),
)

INFO:minirag:Logger initialized for working directory: resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data
INFO:minirag:Load KV llm_response_cache with 183 data
INFO:minirag:Load KV full_docs with 88 data
INFO:minirag:Load KV text_chunks with 88 data
INFO:minirag:Loaded graph from resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data/graph_chunk_entity_relation.graphml with 281 nodes, 84 edges
INFO:nano-vectordb:Load (263, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data/vdb_entities.json'} 263 data
INFO:nano-vectordb:Load (263, 1024) data
INFO:nano-vectordb:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': 'resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data/vdb_entities_name.json'} 263 data
INFO:nano-vectordb:Load (84, 1024)

INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 14 vectors to entities
INFO:minirag:Inserting 14 vectors to entities_name
INFO:minirag:Inserting 8 vectors to relationships


INFO:minirag:Writing graph with 288 nodes, 91 edges
INFO:minirag:[New Docs] inserting 4 docs
INFO:minirag:[New Chunks] inserting 4 chunks
INFO:minirag:Inserting 4 vectors to chunks
INFO:minirag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:minirag:Inserting 15 vectors to entities_name
INFO:minirag:Inserting 6 vectors to relationships
INFO:minirag:Writing graph with 398 nodes, 135 edges
INFO:minirag:[New Docs] inserting 1 docs
INFO:minirag:[New Chunks] inserting 1 chunks
INFO:minirag:Inserting 1 vectors to chunks
INFO:minirag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:12239/v1/chat/completions "HTTP/1.1 200 OK"


INFO:minirag:Inserting 3 vectors to entities
INFO:minirag:Inserting 3 vectors to entities_name
INFO:minirag:Inserting 2 vectors to relationships


INFO:minirag:Writing graph with 399 nodes, 136 edges


✅ 异步 RAG 索引构建完成，共插入 137 个 chunk。索引保存在：resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data


MiniRAG(working_dir='resources/data/ablation_rag/financebench_zh/sim@q25_b3_sft_sum@q25_b3_base_index/rag_data', kv_storage='JsonKVStorage', vector_storage='NanoVectorDBStorage', graph_storage='NetworkXStorage', log_level=0, chunk_token_size=1200, chunk_overlap_token_size=100, tiktoken_model_name='gpt-4o-mini', entity_extract_max_gleaning=1, entity_summary_to_max_tokens=500, node_embedding_algorithm='node2vec', node2vec_params={'dimensions': 1536, 'num_walks': 10, 'walk_length': 40, 'window_size': 2, 'iterations': 3, 'random_seed': 3}, embedding_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x7dff40203d00>, embedding_batch_num=32, embedding_func_max_async=16, llm_model_func=<function limit_async_func_call.<locals>.final_decro.<locals>.wait_func at 0x7dff381ecca0>, llm_model_name='base', llm_model_max_token_size=500, llm_model_max_async=16, llm_model_kwargs={}, vector_db_storage_cls_kwargs={}, enable_llm_cache=True, addon_params={}, convert_response_to_